In [45]:
import pandas as pd
import numpy as np
from functools import reduce
from surprise import Dataset, Reader, accuracy, SVD, NMF
from surprise.model_selection import train_test_split
import surprise.prediction_algorithms.knns as knns
import surprise.prediction_algorithms.matrix_factorization
from sklearn.metrics import recall_score

In [3]:
# We load all the variables we care about
ratings= pd.read_csv("ml-100k/ratings.csv")
movies = pd.read_csv('ml-100k/movies.csv')
tags = pd.read_csv('ml-100k/tags.csv')
links = pd.read_csv('ml-100k/links.csv')


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
reader = Reader(rating_scale=(1,5))
# Train/Test split using pandas alone
#train = ratings.sample(frac=0.75, random_state=1234)
#test = ratings.drop(train.index)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=1234)
algo = knns.KNNBasic(sim_options={"name": "pearson"})
predictions = algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
#trainset, testset = train_test_split(ratings, test_size=0.25, random_state=1234)

AttributeError: 'DataFrame' object has no attribute 'raw_ratings'

In [51]:
algo = knns.KNNBasic(sim_options={"name": "pearson"})
algo.fit(trainset)
predictions = algo.test(testset)
predictions

Computing the pearson similarity matrix...
Done computing similarity matrix.


[Prediction(uid=68, iid=166528, r_ui=4.5, est=3.82324478637288, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=432, iid=1221, r_ui=4.0, est=4.4685776059333016, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=325, iid=1968, r_ui=3.0, est=3.783479906361247, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=133, iid=150, r_ui=3.0, est=3.945997452813432, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=187, iid=2502, r_ui=4.5, est=4.195826508503563, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=21, iid=168252, r_ui=3.0, est=4.257519992781679, details={'actual_k': 7, 'was_impossible': False}),
 Prediction(uid=534, iid=83134, r_ui=4.5, est=4.577291946189827, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=387, iid=3168, r_ui=4.0, est=3.7821452147771284, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid=610, iid=103075, r_ui=2.5, est=3.5152826132439556, deta

In [ ]:
# zip is sort of its own inverse, since is variadic function
# you cana call it with all the pairs as arguments and zip them
# creating a single pair with the contents unzipped
def get_recall_score(predictions):
    y_pred, y_true = zip(*[[(p.est > 4), (p.r_ui > 4)] for p in predictions])
    return recall_score(y_true, y_pred)

In [57]:
def groupby_id(d, p):
    if p.uid in d:
        d[p.uid].append(p)
        return d
    else:
        d[p.uid] = [p]
        return d

In [59]:
recomendation_list = reduce(groupby_id, predictions, {})

In [60]:
def truncate_to_k(l, k=10):
    if len(l) > 10:
        return l[0:10]
    else:
        return l

In [61]:
def truncate_recomendation_list(r, k=10):
    r.update((key, truncate_to_k(val, k)) for key, val in r.items())

In [63]:
truncate_recomendation_list(recomendation_list)
recomendation_list

{68: [Prediction(uid=68, iid=166528, r_ui=4.5, est=3.82324478637288, details={'actual_k': 13, 'was_impossible': False}),
  Prediction(uid=68, iid=3253, r_ui=4.5, est=3.623973085927816, details={'actual_k': 36, 'was_impossible': False}),
  Prediction(uid=68, iid=3671, r_ui=3.5, est=3.719447951338336, details={'actual_k': 32, 'was_impossible': False}),
  Prediction(uid=68, iid=170, r_ui=2.5, est=3.3985005783648807, details={'actual_k': 21, 'was_impossible': False}),
  Prediction(uid=68, iid=637, r_ui=2.5, est=2.905505925822118, details={'actual_k': 18, 'was_impossible': False}),
  Prediction(uid=68, iid=26, r_ui=3.0, est=3.4084635491585016, details={'actual_k': 8, 'was_impossible': False}),
  Prediction(uid=68, iid=122922, r_ui=3.0, est=3.7642170293490866, details={'actual_k': 9, 'was_impossible': False}),
  Prediction(uid=68, iid=514, r_ui=2.5, est=3.39771390046449, details={'actual_k': 7, 'was_impossible': False}),
  Prediction(uid=68, iid=6281, r_ui=2.0, est=3.307048786131529, details

In [ ]:

knn_prod = knns.KNNBasic(sim_options={"name": "pearson", 'user_based': False})
knn_prod.fit(trainset)
knn_prod_pred = knn_prod.test(testset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
svd = SVD()
svd.fit(trainset)
svd_predictions = svd.test(testset)
uuid= []
[uuid.append(pred.uid) for pred in svd_predictions]
len(uuid)/ len(set(uuid))

41.32622950819672

In [8]:
nmf = NMF()
nmf.fit(trainset)
nmf_predictions = svd.test(testset)

NameError: name 'svd' is not defined

In [ ]:
[movies[movies['movieId'] == recomended_id]['title'] for recomended_id in [item.iid for item in svd_predictions[0:5]]]

[9433    Rogue One: A Star Wars Story (2016)
 Name: title, dtype: object,
 922    Godfather: Part II, The (1974)
 Name: title, dtype: object,
 1445    Breakfast Club, The (1985)
 Name: title, dtype: object,
 123    Apollo 13 (1995)
 Name: title, dtype: object,
 1883    Office Space (1999)
 Name: title, dtype: object]

In [ ]:
def get_titles(prediction, n, movies):
    return [movies.loc[movies['movieId'] == recomended_id, 'title'].item() for recomended_id in [item.iid for item in prediction[0:n]]]

In [ ]:
import itertools

#[algo.predict(uid, iid) for uid, iid in (itertools.product([test[0] for test in testset], movies['movieId']))]

KeyboardInterrupt: 

In [ ]:
get_titles(predictions, 30, movies)

['Rogue One: A Star Wars Story (2016)',
 'Godfather: Part II, The (1974)',
 'Breakfast Club, The (1985)',
 'Apollo 13 (1995)',
 'Office Space (1999)',
 'Logan (2017)',
 'Tucker & Dale vs Evil (2010)',
 'Easy Rider (1969)',
 'Purge, The (2013)',
 'Guardians of the Galaxy (2014)',
 'Dreamers, The (2003)',
 'Congo (1995)',
 'Talented Mr. Ripley, The (1999)',
 'Scanner Darkly, A (2006)',
 'Get Shorty (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Dogtown and Z-Boyz (2001)',
 'Fight Club (1999)',
 'Hard Candy (2005)',
 'Bruce Almighty (2003)',
 'Chamber, The (1996)',
 'American President, The (1995)',
 'Mission: Impossible (1996)',
 'Forrest Gump (1994)',
 'Taxi 4 (2007)',
 'Snowpiercer (2013)',
 'Truman Show, The (1998)',
 'Desperately Seeking Susan (1985)',
 'Twister (1996)',
 'Beauty and the Beast (1991)']

In [ ]:
get_titles(svd_predictions, 30, movies)

['Rogue One: A Star Wars Story (2016)',
 'Godfather: Part II, The (1974)',
 'Breakfast Club, The (1985)',
 'Apollo 13 (1995)',
 'Office Space (1999)',
 'Logan (2017)',
 'Tucker & Dale vs Evil (2010)',
 'Easy Rider (1969)',
 'Purge, The (2013)',
 'Guardians of the Galaxy (2014)',
 'Dreamers, The (2003)',
 'Congo (1995)',
 'Talented Mr. Ripley, The (1999)',
 'Scanner Darkly, A (2006)',
 'Get Shorty (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Dogtown and Z-Boyz (2001)',
 'Fight Club (1999)',
 'Hard Candy (2005)',
 'Bruce Almighty (2003)',
 'Chamber, The (1996)',
 'American President, The (1995)',
 'Mission: Impossible (1996)',
 'Forrest Gump (1994)',
 'Taxi 4 (2007)',
 'Snowpiercer (2013)',
 'Truman Show, The (1998)',
 'Desperately Seeking Susan (1985)',
 'Twister (1996)',
 'Beauty and the Beast (1991)']

In [ ]:
get_titles(knn_prod_pred, 10, movies)

['Rogue One: A Star Wars Story (2016)',
 'Godfather: Part II, The (1974)',
 'Breakfast Club, The (1985)',
 'Apollo 13 (1995)',
 'Office Space (1999)',
 'Logan (2017)',
 'Tucker & Dale vs Evil (2010)',
 'Easy Rider (1969)',
 'Purge, The (2013)',
 'Guardians of the Galaxy (2014)']

In [ ]:
get_titles(nmf_predictions, 10, movies)

['Rogue One: A Star Wars Story (2016)',
 'Godfather: Part II, The (1974)',
 'Breakfast Club, The (1985)',
 'Apollo 13 (1995)',
 'Office Space (1999)',
 'Logan (2017)',
 'Tucker & Dale vs Evil (2010)',
 'Easy Rider (1969)',
 'Purge, The (2013)',
 'Guardians of the Galaxy (2014)']